In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
from astropy.cosmology import FlatLambdaCDM

In [2]:
# ----------------------------------
# Constants & cosmology
# ----------------------------------
H0, OM0   = 70.0, 0.3
SIGMA_INT = 0.10
LN10  = np.log(10.0)
COSMO = FlatLambdaCDM(H0=H0, Om0=OM0)

def mu_theory(z):
    """Theoretical distance modulus mu_th(z) in mag via Astropy."""
    return COSMO.distmod(z).value

CSV_PATH = "/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv"

In [ ]:
# ----------------------------------
# I/O with cleaning
# ----------------------------------
def load_ztf_csv(path):
    """
    Loads the ZTF-style CSV and removes rows with missing light-curve parameters
    or redshift. Keeps everything else untouched.
    """
    df = pd.read_csv(path)
    df = df.dropna(subset=["x0", "x1", "c", "redshift"]).copy()
    return df.reset_index(drop=True)

In [ ]:
# ----------------------------------
# Error propagation for mu_obs
# ----------------------------------
def sigma_mu_meas2(alpha, beta, x0, x1, c,
                   sig_x0, sig_x1, sig_c,
                   cov_x0_x1, cov_x0_c, cov_x1_c):
    """
    Propagate (x0, x1, c) measurement covariances into var(mu_obs):
      mu_obs = -2.5*log10(x0) + alpha*x1 - beta*c + M_B
    """
    dmu_dx0 = -2.5 / (x0 * LN10)
    dmu_dx1 = +alpha
    dmu_dc  = -beta

    var = (dmu_dx0**2) * (sig_x0**2) \
        + (dmu_dx1**2) * (sig_x1**2) \
        + (dmu_dc**2)  * (sig_c**2)  \
        + 2.0 * (dmu_dx0*dmu_dx1) * cov_x0_x1 \
        + 2.0 * (dmu_dx0*dmu_dc)  * cov_x0_c  \
        + 2.0 * (dmu_dx1*dmu_dc)  * cov_x1_c

    return np.clip(var, 0.0, None)

# ----------------------------------
# Chi^2 with analytic M_B
# ----------------------------------
def chisq_alpha_beta(theta, z, x0, x1, c,
                     sig_x0, sig_x1, sig_c,
                     cov_x0_x1, cov_x0_c, cov_x1_c,
                     sigma_int=SIGMA_INT):
    """
    Minimize over (alpha, beta). For each (alpha,beta), solve M_B analytically:
      chi2 = sum [ (mu_star + M_B - mu_th)^2 / s2 ]
      d(chi2)/dM_B = 0  =>  M_B = sum( (mu_th - mu_star)/s2 ) / sum(1/s2)
    """
    alpha, beta = theta
    mu_th  = mu_theory(z)
    mu_star = -2.5*np.log10(x0) + alpha*x1 - beta*c

    s2_meas = sigma_mu_meas2(alpha, beta, x0, x1, c,
                             sig_x0, sig_x1, sig_c,
                             cov_x0_x1, cov_x0_c, cov_x1_c)
    s2 = s2_meas + sigma_int**2
    w  = 1.0 / np.clip(s2, 1e-12, None)

    MB_hat = np.sum((mu_th - mu_star) * w) / np.sum(w)
    resid  = (mu_star + MB_hat) - mu_th
    chi2   = np.sum(resid**2 * w)

    # Cache for retrieval after minimize()
    chisq_alpha_beta.last_MB   = MB_hat
    chisq_alpha_beta.last_chi2 = chi2
    chisq_alpha_beta.last_w    = w
    chisq_alpha_beta.last_resid= resid
    return chi2

# ----------------------------------
# Driver
# ----------------------------------
def fit_tripp(csv_path=CSV_PATH, alpha0=0.14, beta0=3.1,
              method="Nelder-Mead"):
    df = load_ztf_csv(csv_path)

    z   = df["redshift"].to_numpy()
    x0  = df["x0"].to_numpy()
    x1  = df["x1"].to_numpy()
    c   = df["c"].to_numpy()

    sig_x0 = df["x0_err"].to_numpy()
    sig_x1 = df["x1_err"].to_numpy()
    sig_c  = df["c_err"].to_numpy()

    cov_x0_x1 = df["cov_x0_x1"].to_numpy()
    cov_x0_c  = df["cov_x0_c"].to_numpy()
    cov_x1_c  = df["cov_x1_c"].to_numpy()

    # Minimize chi2(alpha, beta); MB is analytic
    x0_guess = np.array([alpha0, beta0], dtype=float)
    res = optimize.minimize(
        chisq_alpha_beta, x0_guess,
        args=(z, x0, x1, c, sig_x0, sig_x1, sig_c, cov_x0_x1, cov_x0_c, cov_x1_c),
        method=method,
        options={"maxiter": 5000, "xatol": 1e-8, "fatol": 1e-8}
    )

    alpha_hat, beta_hat = res.x
    MB_hat   = chisq_alpha_beta.last_MB
    chi2_min = chisq_alpha_beta.last_chi2
    dof = len(df) - 2  # (alpha,beta) free; MB solved analytically

    out = {
        "alpha": alpha_hat,
        "beta":  beta_hat,
        "MB":    MB_hat,
        "chi2":  chi2_min,
        "dof":   dof,
        "chi2_dof": chi2_min / max(dof, 1),
        "success": res.success,
        "message": res.message,
        "n_used": len(df)
    }
    return out

# ----------------------------------
# Optional: quick test if run as a script
# ----------------------------------
if __name__ == "__main__":
    results = fit_tripp("/Users/pittsburghgraduatestudent/repos/myc21_ztf_mu/ZTF_snia_data.csv")
    print("\nBest-fit parameters:")
    for k, v in results.items():
        print(f"{k:10s}: {v}")


Best-fit parameters:
alpha     : 0.16543916537421038
beta      : 3.6208289771289097
MB        : 29.768179155406138
chi2      : 16748.714199109403
dof       : 3574
chi2_dof  : 4.686265864328316
success   : True
message   : Optimization terminated successfully.
n_used    : 3576
